In [2]:
# ! pip install Selenium

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import re
from time import sleep
import pandas as pd

In [4]:
# 크롬으로 국민청원 사이트 접속
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
url = 'https://www1.president.go.kr/petitions'
driver.get(url)


In [9]:
# 키워드 검색
keyword = '균형발전대책'  # test 위한 키워드
driver.find_element_by_css_selector('#LIST_search').send_keys(keyword)
driver.find_element_by_css_selector('#cont_view > div.cs_area > div > div:nth-child(3) > form > button').click()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#LIST_search"}
  (Session info: chrome=70.0.3538.102)
  (Driver info: chromedriver=2.43.600210 (68dcf5eebde37173d4027fa8635e332711d2874a),platform=Windows NT 10.0.17134 x86_64)


In [10]:
# 날짜 지정 후 재검색
driver.find_element_by_css_selector('#start_Date').send_keys('2018/01/01')
driver.find_element_by_css_selector('#end_Date').send_keys('2018/10/31')
driver.find_element_by_css_selector('#login > li.search_PGbox_btn > input[type="image"]').click()

In [29]:
# 소스 추출
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')

In [33]:
# 검색개수 및 페이지 수 계산
result_num = soup.find('div', 'searchPG_title').find('span', 'searchPG_allNum').get_text().replace('(', '').replace('건)', '').replace(',', '')
pages = int(result_num) // 10
pages

149

In [22]:
# 크롤링 변수 설정
contents = soup.find_all('div', 'PG_contens_pd_list')
title = contents[0].find('h2').get_text()
start_date = contents[0].find_all('li')[0].find('span').get_text()
people_agreed = contents[0].find_all('li')[2].find('span').get_text()
link = 'https://www1.president.go.kr' + contents[0].find('a')['href']
# title, start_date, people_agreed, link

('세가지 제안', '2018-10-25', '3', 'https://www1.president.go.kr/petitions/419894')

In [41]:
# 페이지 버튼 눌러보기
button_num = 3
selector_value = '#contents > div.search_PG_contens > div > div.paging > div > div:nth-child(2) > a:nth-child({button_num})'.format(button_num = button_num)
driver.find_element_by_css_selector(selector_value).click()

In [3]:
'''
전체 크롤링
'''
# 기본 변수 지정
add_title = []
add_start_date = []
add_people_agreed = []
add_link = []
page_num = 1
button_num = 1

# 페이지 접속 후 키워드 검색
driver_path = "driver/chromedriver.exe"
driver = webdriver.Chrome(executable_path=driver_path)
url = 'https://www1.president.go.kr/petitions'
driver.get(url)
keyword = '저출산'
driver.find_element_by_css_selector('#LIST_search').send_keys(keyword)
driver.find_element_by_css_selector('#cont_view > div.cs_area > div > div:nth-child(3) > form > button').click()
driver.find_element_by_css_selector('#start_Date').send_keys('2018/01/01')
driver.find_element_by_css_selector('#end_Date').send_keys('2018/10/31')
driver.find_element_by_css_selector('#login > li.search_PGbox_btn > input[type="image"]').click()
# 소스 가져온 후 검색 개수, 페이지 수 계산
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
result_num = soup.find('div', 'searchPG_title').find('span', 'searchPG_allNum').get_text().replace('(', '').replace('건)', '').replace(',', '')
pages = int(result_num) // 10 + 1
print('result_num', result_num, 'pages', pages)

# Main
while True:
    # 소스 저장, soup 변환
    print('page_num', page_num, 'button_num', button_num)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    contents = soup.find_all('div', 'PG_contens_pd_list')
    
    # 크롤링
    for tag in contents:
        title = tag.find('h2').get_text()
        print('title', title)
        start_date = tag.find_all('li')[0].find('span').get_text()
        people_agreed = tag.find_all('li')[2].find('span').get_text()
        link = 'https://www1.president.go.kr' + tag.find('a')['href']

        add_title.append(title)
        add_start_date.append(start_date)
        add_people_agreed.append(people_agreed)
        add_link.append(link)
        sleep(0.5)
        
    # 페이지 이동
    if page_num == pages:
        print('Crawling Finished!!!')
        break
    else:
        if button_num == 7:  # click Next button
            driver.find_element_by_css_selector('#contents > div.search_PG_contens > div > div.paging > div > div.p_btn.next > a').click()
            page_num += 1
            button_num = 1
        else:  # click next page number
            button_num += 1
            selector_value = '#contents > div.search_PG_contens > div > div.paging > div > div:nth-child(2) > a:nth-child({button_num})'.format(button_num = button_num)
            driver.find_element_by_css_selector(selector_value).click()
            page_num += 1
            


result_num 3339 pages 334
page_num 1 button_num 1
title 저출산 관련 육아휴직 및 어린이집 증설
title 저출산 ↔ 어린이집 부족. 서로 너무 상충되는 관계 아닌가요?
title 어린이집을 유지해주세요
title 오늘 안하면 늦습니다.  누구나 마음놓고 출산할 수 있는 제도 마련 시급합니다.
title 가족돌봄휴가제도를 조속히 시행해야 합니다.
title 우리 아이들을 위해서..... 사립유치원 해결 방안 제안합니다.
title 양육비 안주는 나쁜 아빠들
title ★ mbc PD수첩(미친 아파트값의 비밀ㅡ2탄)"대통령은 답하라"
title 부동산정책정말실망입니다
title 비정규직을 없애주세요..
page_num 2 button_num 2
title 신혼희망타운 가점기준 자녀1명당 24만원?
title 한국경제, 행정부가 키우는 국민 불안감 해소해 줘야
title 안녕하세요 대통령님, 혼자아이를키우는 한부모가정입니다.
title 저출산과 일자리문제를 해결하는 방법을 생각해 봤습니다.
title 멀리보고 크게 생각합시다ᆞᆞ
title 불임부부 국가지원 읽어주세요
title 국민연금과 관련하여 궁금증 및 제안합니다.
title 저도 전세살이지만, 부동산 정책은 지금이라도 멈춰야 합니다.
title 청량리 경동 시장 경제 활성화
title 사명감을 가지고 일하는 국제결혼회사 불법영업회사들을 엄벌하여주시길 바랍니다.
page_num 3 button_num 3
title 사립유치원 없애고 유시민작가님 아이디어를 채택해주세요
title 무주택자 요건 완화해야하는거 아닙니까?
title 여성가족부의 사과와 해체 요구
title 성에 대한 이식을 좋은쪽으로 발전
title 전국의 임산부 배려석에 임산부임을 확인 할 수 있는 기기 설치
title 대한민국 대통령이 대한민국에 기생하는 조선족들을 추방시켜야 하는 이유 세 가지.
title 난임부부정부지원
title 어쩌다 이나라가 국민보다는 외노자를 위한 나라가 되었나요
title 

title 어린이집, 사립 유치원에 대한 현실적인 정책을 마련해 주세요
title 이게 사는겁니까?
title 다자녀 혜택 ( 사망신고한 형제자매 포함해주세요!)
title 진보의 오류
title 화물차
page_num 25 button_num 4
title 저소득층의 고민
title 저는 시간선택제 공무원입니다.
title 모자가정 어떻게 살아가라는 것인지요ㅠㅠ
title 음주사건은 감형이아니라,  가중처벌로
title 난임시술, 시험관아기 하는 여성분들도 실업급여 받을수 있도록 해주세요...
title 저출산문제 만큼 중요한 100세시대의 노인봉양문제
title 전국 병원 지문인식 시스템 도입 + 외국인 건강보험 직장가입자 한정
title 20대남  혼인신고와  신생아 출생신고  마치면  군대 면제
title 어린이집들에게  급식관리지원센터에게까지 관리받아야 합니까
title 자녀장녀금문의
page_num 26 button_num 5
title 콜센터도 탄력근무제 시행해 주세요
title 롯데월드 같은 시설에 아빠가 아기를 돌볼 수 있는 시설을 의무화해주세요
title 신혼부부행복주택평수가작네요~애낳지말고살라는건가요
title 주거용오피스텔 취득세 인하부탁드립니다..ㅠ.ㅠ
title 다자녀 가정이 행복한 나라
title 환경 좀 지켜주세요 끝까지 제발 읽어주세요 여러분들의 미래 후손 동물의 미래가 달린 일입니다
title 상속제 폐지로 경제민주화를 이뤄야 나라가 삽니다.
title 아기를 꼭 낳고싶어요
title 서울에 신혼희망타운 좀 지어주세요!!!!!!!!! 부동산, 집값!!
title 용인시 교육청은 왜 학생배치를 안해주는가
page_num 27 button_num 6
title 한국은행총재를 교체하세요  이유있어요 여러분 읽어주세요
title 청소년법 폐지
title -저 출산 대책-
title 불합리한 비자정책
title 부모 부양의무 강화해야 출산율 늘어납니다 .
title 출산율이 높은 지역에 더 많은 육아지원이 필요하다고봅니다.
title 다

title 저출산 문제와 최근 부동산
title 결혼예정자 및 신혼부부에게 59m2의 주택을 20~30년 상황조건으로 정부(LH)가 의무적으로 공급하자!
page_num 49 button_num 7
title 현 시점에서 부동산. 청년실업에 드리고 싶은 제안
title 증여세, 상속세 폐지 바랍니다.
title 저출산,다자녀지원정책 및 서민지원대책방안 건의
title 저출산 대책
title 수도권 집값 안정 및 저출산 해소 방안
title 저출산 시대 직장인 임산부 근무환경
title 부모가 마음편하게 일할수있는 나라
title (개인회생)모두를 위한 나라, 다 함께 잘 사는 포용국가 청와대 대통령비서실 2018.09.06
title 어린이집 왜 안바뀌나요..
title 양극화가심화되고빈부격차가심해지는이때 우리는 부자털어서가난구제햔홍길동이 그립다.
page_num 50 button_num 1
title 저출산 대책-2인 또는 다자녀 자녀들에게 특례입학허용
title 도대체 누구를 위한 대책인가요...
title 퀴어축제를 막아주세요!!!!!!!!!!!!!!!!
title 무엇이든 최상의 방법은 있습니다
title 공공 임대주택  많이 짖는것만이 부동산 정책(투기억제) 성공의  길이다,
title 부동산정책및 저출산 해소 대책
title 어린이집 입소대기조건이 변경되어야 합니다.
title 限많은 이산가족들......
title 전국토를 투기장으로 만들지말고,이젠 금리 올려주세요~
title 고용노동부 장관님 바뀌셨으니 어린이집들 감사 방식을 바꾸어 주시기 바랍니다 너무 힘듭니다
page_num 51 button_num 2
title ★★★이번에 발표될 부동산카드ㅡ정권의 명운이 달렸다(사즉생)
title 무능한 청와대
title 용산을 보는 다른 시각...저출산해결책...가족공원?  삶의터전!
title 자녀장려금 심사조건을 확장해주세요!
title 그린벨트풀지말고 금리를 인상하시라구요
title 여성도 군 복무를
title 민생이나 챙기십시오.
title 고등

title 저출산대책으로 의무 휴직 기간 2년이상 연장 부탁드립니다
title 어린이집 원장의 최저임금을 확립해주세요.
title 신혼부부 ?  구혼부부?
title 산부인과 전문의들의 낙태수술 중단선언을 해제 해주십시오
title 다문화 에대해서
page_num 73 button_num 3
title 저출산은.나라기둥흔든다.결코 작은일아니다
title 전세자금대출 규제라뇨!!!???대출상환 다해서 둘째아이 낳고 살집 마련하고 행복하게 살고 싶어요
title 저출산에 대한 의견을 올릴까합니다
title 주택보유제한으로 대한민국을 살려주세요
title 초등생도 아는 해법 보유세와 금리인상만이 답입니다.
title 당신들을 미래를 버리는가??
title 서울아빠트는 콩크리트가아니라금덩어리다
title 새정부탄생후폭등한 서울집값환원 집에서민들이죽는다
title 부동산잡는것 간단해요
title 출산장려금 1억 + 자녀수에 따른 소득세 차등
page_num 74 button_num 4
title 저출산 대책으로 집을 주세요!!!!!!!!!!!!!!!
title 범죄,탈세와의 전쟁 선포해주세요
title 개헌하자
title 조선족동포들 대한 민국 의무병역참가하면 영주권과국적주어라
title 저 출산은 정치에 좌지우지 하지 않는 장기적인 국가 프로젝트로
title 제안 2) 저출산 대책=  필요시 24시간 또는 며칠간  이용 할수 있는국가 또는 지자체 운영  유아 보호및 관리  시설 조성
title 1)어린이집 차량 사고방지 등하원 확인용 지문인식,카드시스템 적용
title 저출산 인구줄이기 말고는 답도 해법도 없다
title 부동산투기 공화국/해결책은 오른만큼 양도세부과
title 부동산가격 폭등을 잡아야 저출산 대책 극복할 수 있습니다. 보유세 인상하시길 간곡히 부탁드립니다.
page_num 75 button_num 5
title 문 정부 15개월 잔치는 끝났다.  (청와대는 이 기사를 보고 정신차리는 기회로)
title (부동산) 가구합산소득 7천만원 이상 전세대출 

title 저출산으로 인구는감소하는데왜일자리가없다고난리냐?
title 저출산폭넓게투자해야합니다
title 신뢰를 잃은 국민연금은 더이상 존재될 명분이 없다.
title 이미 고갈된 공무원연금, 사학연금, 군인연금 소득대체율 30%로 개혁하라.
title 저출산 대책
title 갈등부채질의 국민연금을 폐지만이 살길입니다.
title 여성가족부
title 국민연금 단계적 폐지
title 일자리쇼크 등 불황은 근본원인은 초저출산
page_num 97 button_num 6
title 대통령님이 직접나서주세요  올해최대폭상승 서울집값
title 저출산 대책과 연금, 두마리 토끼를 잡기
title 귀 담지 않은 무능한 정부(국민연금)
title 저는 43살에 임산부입니다
title 임신 의료지원 및 출산비용 지원해주세요.
title 국민연금  폐지
title 무슬림난민과 독립운동가하고 비교하지마라.
title 결혼도안하고 아이도 안 낳으면 세금폭탄 이나 벌금 안겨라
title 도시가스 전기세도  카드로납부
title 다자녀부모에게 노후연금주자 (국민연금,저출산,고령화에 대한 가장 현실적인 방법)
page_num 98 button_num 7
title 예전 정부나 현 정부는 국민이 무엇을 원하는지 모르는 듯 합니다.
title 3세 영유아 가정 전기세인하. 여름 방학맞은 학생 가정은?
title 경기파탄 및 일자리
title 4년제 대학을 대폭 줄이고 전문대, 특성화고 활성화 해야 합니다
title 국민연금개편한다면 관리공단 개혁도 필히 해주세요.
title 밑빠진독 국민연금 폐지 하고 일괄 일시금수령
title 국민연금 폐지하자
title 어린이집 아동학대
title 저출산 걱정보다 일단 결혼부터 할수있게 해주세요~
title 국민연금 개편안 제안
page_num 99 button_num 1
title 아동보육수당
title 대통령님 청년취업 및 실업율 악화 원인과 이유 입니다
title 국민을 생각하시긴 하는겁니까
title 저출산 해결방안
title 80년도생이보는

...

title (내용추가)헌법개정 반대합니다.! 일반행정공무원과 운전공무원을 폐지해야 이 나라가 잘 살아갑니다.!
title 소 잃고 외양간 고치는 대한민국 ( 사이비종교 )
title 아파트 특별공급.. 저출산시대 다자녀 가구를 위한 청원
title 민간어린이집과 가정어린이집 원장들도 고용보험을 가입하게 정책을 바꾸어 주시기 바랍니다
title 상속세 폐지 청원
title 근로자 휴가 지원사업
page_num 237 button_num 6
title 다자녀 특별공급을 왜 없애는 걸까요
title (내용추가)헌법개정 반대합니다.! 일반행정공무원과 운전공무원을 폐지해야 이 나라가 잘 살아갑니다.!
title 다자녀혜택이 뭔가요  ?
title 국민들이 생각하는 저출산 이유는~~~~~~~~~~~~^^^^^
title 저출산,고령화 대책
title 결혼과 저출산의 문제~~~~~^^
title 언론과 재벌의 유착 관계를 철저히 조사 해 주십시오.
title 최근 뉴스를 장식하는 재벌2ᆞ3세들 모두 불러서 국회청문회를 실시해라ᆞ 국민의 이름으로
title (내용추가)헌법개정 반대합니다.! 일반행정공무원과 운전공무원을 폐지해야 이 나라가 잘 살아갑니다.!
title 납세 안하면 투표권 안주는 입법청원
page_num 238 button_num 7
title 국가재정 70%를 벌금 , 30%를 세금으로
title 문재인 대통령님 솔직히 이나라에서 남자 여자 싸우는거 관심없으시죠
title 납세액 비례하여 투표권 차등 지급
title 여성가족부가있는한 미래는없다
title '워킹맘'의 멸종, 저출산 해결의 핵심고리
title 저출산문제
title 도대체 저출산 문제 어떻게 해결할 겁니까?
title 어린이집및유치원 폭력 신고체계 및 처벌 강화 지침을 정확하게 만들어주세요.
title 대중교통 임산부석에 대한 대책이 필요합니다
title 어린이집 아동학대범 처벌에 대해..
page_num 239 button_num 1
title 대한항공과 삼성전자의 불의
title 저출산 해결대

title 저출산에 대한 효과적인 정책(가족수당 정책)
title 모든 기업에 의무적으로 ' 임신했다고 눈치주지 않기 ' 켐페인을 제안합니다
title 사내 임신 허용제 추진
title 입덧억제제 의료보험 적용 시켜 주셍ᆢㄷ
title 굴러온돌이 박힌돌 빼내는 다문화정책 원점 재검토 해야합니다
title 다둥이 기준을 두자녀 이상으로 바꾸어야 합니다.
title 임산부 배려석에 임산부는 앉을수가 없네요!
title 저출산 시대 - 출산 장려 차원에서 출산 가정에 에너지 지원금 지급을 검토 바랍니다.
title (내용추가)헌법개정 반대합니다.! 일반행정공무원과 운전공무원을 폐지해야 이 나라가 잘 살아갑니다.!
page_num 260 button_num 1
title 육아정책
title 암적 존재, 뛰는 다주택자 위에 나는 건물주(건물사냥꾼)
title 저출산 대책
title 헌법개정 반대합니다.! 일반행정공무원과 운전공무원을 폐지해야 이 나라가 잘 살아갑니다.!
title 다자녀(3자녀이상)복지혜택을 1자녀,2자녀에게도 차등적으로 적용해 주세요~~!!!
title 시험관,출산까지의 비용
title (재청원)남성만의 실질적 독박 국방의무 이행에서 벗어나 여성도 의무 이행에 동참하도록 법률개정이 되어야 합니다.
title 출산 지원금 차별
title 지하철임산부고통
title 학교 공기청정기의무화
page_num 261 button_num 2
title 폐기물처리장이라니요..  제발 아이들이 맘껏 뛰어놀  수 있게 해주세요.
title 저출산
title 법정 근로시간을 주 35시간으로 줄여주세요.
title 근육병 어린이 환자 들을 도와주세요
title 세계최저출산 나라풍전등화 과감한 교육개혁만이 저출산해결한다
title 초 4년 중 2, 고 2, 대학 2년으로 학제 개편
title 저출산.남녀평등을 모두해결할수 잇는방법
title 토지공개념 도대체 뭐하는건가요?
title 존경하는 하느님 믿는 가톨릭 디모데오 문대통령님 상서
title 대한민국 이대로가면 안됩니다

title 네 아이의 아빠입니다
title 중소기업에 다니는 사람들은 다 죽으라는 얘기인가요?
page_num 284 button_num 4
title 임신초기 여성의 휴직제도 강제규정화를 청합니다.
title 동성애 반대
title 여성 국방 의무 당장 시행하라
title 저출산 대책을  공무원이  세우니   나라가 망한다
title 아이 낳고 싶어도 낳지 못하는 사람도 많습니다.
title 주4일제 근무 제안합니다.
title 한부모가정에대한지원을늘려주세요...
title 대학생 과제총량제를 제안합니다.
title 근로시간 단축과 관련된 실효성이 제대로 이루어질 수 있도록 고용노동부의 철저한 관리 감독에 대해 청원합니다
title 명절증후군은 반복됩니다. 명절을 폐지해주세요.
page_num 285 button_num 5
title 신혼부부 3억까지 무이자 20년 해주면 저출산 해결됩니다.
title 내가 . 내 아이가 살아가도 안전한 나라입니까?
title 내가. 또는 내 아이가 살기 좋은 나라입니까?
title 저출산고령화인력난문제.
title 저출산 문제
title 저출산 해결을 위한 근본적 원인 집값상승문제 해결
title 저출산ㅡ정말 여성들이 원하는 정책
title 저출산 해결책!(수정본)
title 26살 평범한 청년이 생각하는 저출산/취업문제에 관한 생각입니다. 국정운영에 도움이 되었으면 좋겟네요...많은 분들의 공감 부탁드립니다.
title 저출산..대한민국 사라집니다..
page_num 286 button_num 6
title 초 저출산 시대 엄마 수당으로 엄마들의 긍지 및 자부심을 느낄수 있도록 급여 및 수당이 필요함을 청원 드립니다.
title 남자는 군대를 가니 여자는 국방세
title 외국여성 특별이민법 새로 만들어주세요.
title 기초.광역의원 증원 거부권행사 해주세요.
title 일자리,  집값, 저출산
title 국방의무 대상은 "남자"가 아닌 "모든국민"이다
title 출산관련 지원 정책을 늘려주셨으면 합니다
title 저출

title 국회의원임금을 최저시급으로
title 중학생의 고령화 사회와 저출산의 문제에 대한 청원
title 공무원들부터 잘못된 회식문화 청산에 앞장설 수 있도록 권고나 지침 조례제정이 필요합니다.
title 선거연령 하향을 만 16세부터 해주시길 청원합니다.
title 반일근무제 도입!  청년실업 해소를 위한 방안으로 청원합니다!
title 민간어린이집과 가정어린이집 원장의 입장
title 민간어린이집과 가정어린이집 원장들의 입장
page_num 308 button_num 7
title 사립 유치원 교사 인권문제 (영유아 교사 포함)
title 불쌍한 영어회화전문강사인 아내
title 저녁이 있는삶 + 실업률 낮추기
title 저희24시 어린이집 좀 돌아봐 주세요!!
title 신혼부부 주택마련대출의 비현실적인 연봉7천만원 제한
title 정교사와 기간제교사의 맞춤형복지 불합리한 차별 적용을 철폐해 주세요.
title 저출산 현상에 대한 대책으로 여성 난자 동결을 정책으로 추진해 주세요.
title 개헌에 절대로 성평등이 들어가면 안됩니다. 오직 양성평등 헌법을 수호해야합니다.
title 맞벌이 워킹맘 입니다.
title 강남 고가주택은 취득세를 중과해야 합니다
page_num 309 button_num 1
title 노래방 도우미와 보도방
title 저출산 해결 대책 국민 의견을 들어주세요
title 1가구 2주택부터 부동산  양도세 100% 과세 지금 당장 시행 해주세요!!
title 비정상적인 집값폭등을 바로잡아 주십시오
title 청년 10만인을 판매농부로 양성하십시오.(청년일자리근본적해결책)
title 블록체인&암호화폐의 미래
title 연봉제 최저시급 관련 청원 합니다
title 한부모가정
title 여성 징병제 실시
title 말로만 다자녀 혜택인 코레일!!!
page_num 310 button_num 2
title 저출산대책? 출산비용 좀 낮춰주세요.
title 청년일자리에 대한 아이디어 2가지
title 세계경제학자 한국경제 엄청나게 위험하다

title 청소년 등하교 시간 대중교통 요금 무료화 정책을 만들어주세요.
title 내 말을 무시하는 어리석은 청와대!!
title 아기들과 엄마들을 위한 수유실을 만들어주세요
title 아동성범죄자 처벌강화, 주취 가중처벌, 신상공개, 사형집행 요구합니다.
title 정신과진료를 국민검강검진 항목에 포함시켜주십시오.
title 유치원생 성폭행 또 일어났네요 법 좀 제발 강화해주세요
title [고준희양 사건 관련] 영유아,아동 ,여성  관려 범죄에 대한 슈퍼법률 제안.
title 아동관련 범죄 강력처벌 원합니다.
page_num 332 button_num 3
title '학부모 바우처' 제안 - 봉사 활동에 학부모가 참여하면 교육청에서 바우처를 발행해주는 제도
title 최저임금제를 대체할 방안을 제안합니다.
title 다자녀는 말뿐...
title 저출산 시대, 임신했다고 말씀드렸더니  조용히 나가라고 합니다.제발 도와주십시오...
title 최저시급 계산법 및 물가
title 아동 성범죄자 처벌 강화
title 미성년자 성범죄 처벌 수위 강화
title 아동성범죄자들의 천국이 대한민국이래요
title 끊이지않는 아동성범죄.
title 저출산은 대한민국 미래가 없습니다..
page_num 333 button_num 4
title 배곧신도시 초등학교돌봄교실 학급충원에 대하여
title 아이돌봄서비스/어린이집 (보육) 재정비 및 지원 확대
title 출산 장려 정책
title 중소기업 재직 8개월차... 임신했다고 말씀드렸더니  회사 나가라고 합니다.
title 국정 최우선 순위 저출산 대책
title 현정부의 대한민국 건국일(1919년) 선포에 맞춰 재외자의 한국적 회복을 허용해야 한다
title 민간어린이집과 가정어린이집의 원장의 입장
title 논란이끊이지않는 어린이집학대 더이상두고볼문제는 아닙니다.
title 다자녀 정책을 개선해주세요.
title 인터넷실명제가 꼭 필요한 시점입니다.
page_num 334 button_num 5
title 어린이집 c

In [51]:
add_start_date

['2018-10-31',
 '2018-10-31',
 '2018-10-29',
 '2018-10-29',
 '2018-10-28',
 '2018-10-27',
 '2018-10-27',
 '2018-10-26',
 '2018-10-25',
 '2018-10-25',
 '2018-10-24',
 '2018-10-24',
 '2018-10-24',
 '2018-10-24',
 '2018-10-23',
 '2018-10-22',
 '2018-10-22',
 '2018-10-22',
 '2018-10-22',
 '2018-10-22',
 '2018-10-22',
 '2018-10-22',
 '2018-10-22',
 '2018-10-19',
 '2018-10-19',
 '2018-10-19',
 '2018-10-17',
 '2018-10-16',
 '2018-10-16',
 '2018-10-16',
 '2018-10-16',
 '2018-10-16',
 '2018-10-15',
 '2018-10-15',
 '2018-10-15',
 '2018-10-14',
 '2018-10-14',
 '2018-10-13',
 '2018-10-13',
 '2018-10-12',
 '2018-10-12',
 '2018-10-12',
 '2018-10-12',
 '2018-10-12',
 '2018-10-12',
 '2018-10-12',
 '2018-10-10',
 '2018-10-10',
 '2018-10-10',
 '2018-10-10',
 '2018-10-09',
 '2018-10-08',
 '2018-10-08',
 '2018-10-08',
 '2018-10-07',
 '2018-10-07',
 '2018-10-07',
 '2018-10-07',
 '2018-10-06',
 '2018-10-06',
 '2018-10-06',
 '2018-10-06',
 '2018-10-05',
 '2018-10-05',
 '2018-10-04',
 '2018-10-03',
 '2018-10-

In [52]:
add_people_agreed

['3',
 '4',
 '9',
 '7',
 '22',
 '4',
 '6',
 '302',
 '6',
 '3',
 '39',
 '0',
 '12',
 '1,929',
 '0',
 '171',
 '11',
 '28',
 '15',
 '12',
 '10',
 '3',
 '489',
 '2',
 '6',
 '84',
 '17',
 '3',
 '110',
 '14',
 '4',
 '1,424',
 '49',
 '5',
 '3',
 '5',
 '31',
 '8',
 '9',
 '6',
 '5',
 '9',
 '4',
 '22',
 '896',
 '5',
 '4',
 '11,103',
 '5',
 '1',
 '2',
 '14',
 '4',
 '3',
 '3',
 '28',
 '24',
 '2',
 '16',
 '11',
 '258',
 '1',
 '0',
 '14',
 '4',
 '4',
 '12',
 '448',
 '5',
 '19',
 '5',
 '11',
 '20',
 '3',
 '14',
 '10',
 '124',
 '0',
 '6',
 '1,887',
 '6',
 '17',
 '10',
 '20',
 '12',
 '16',
 '8',
 '1',
 '17',
 '0',
 '7',
 '12',
 '155',
 '21',
 '12',
 '10',
 '0',
 '0',
 '814',
 '3',
 '3',
 '8',
 '12',
 '2',
 '1,834',
 '382',
 '254',
 '11',
 '4',
 '9',
 '11',
 '9',
 '0',
 '1',
 '10',
 '4',
 '13',
 '0',
 '9',
 '1',
 '3',
 '2',
 '4',
 '1',
 '8',
 '2',
 '8',
 '4',
 '0',
 '12',
 '21',
 '5',
 '18',
 '5',
 '7',
 '17',
 '14',
 '148',
 '4',
 '15',
 '2',
 '20',
 '15',
 '7',
 '2',
 '8',
 '2',
 '3',
 '7',
 '13',
 '1

In [53]:
add_link

['https://www1.president.go.kr/petitions/426949',
 'https://www1.president.go.kr/petitions/426738',
 'https://www1.president.go.kr/petitions/423705',
 'https://www1.president.go.kr/petitions/423323',
 'https://www1.president.go.kr/petitions/423175',
 'https://www1.president.go.kr/petitions/422213',
 'https://www1.president.go.kr/petitions/422210',
 'https://www1.president.go.kr/petitions/420989',
 'https://www1.president.go.kr/petitions/419942',
 'https://www1.president.go.kr/petitions/419894',
 'https://www1.president.go.kr/petitions/418403',
 'https://www1.president.go.kr/petitions/418359',
 'https://www1.president.go.kr/petitions/418232',
 'https://www1.president.go.kr/petitions/418211',
 'https://www1.president.go.kr/petitions/417079',
 'https://www1.president.go.kr/petitions/416165',
 'https://www1.president.go.kr/petitions/416119',
 'https://www1.president.go.kr/petitions/416081',
 'https://www1.president.go.kr/petitions/415357',
 'https://www1.president.go.kr/petitions/415284',


In [4]:
df = pd.DataFrame({'제목': add_title,
                   '청원시작': add_start_date,
                   '청원인원': add_people_agreed,
                   'Link': add_link    
})
len(df)

3339

In [5]:
df.head()

,제목,청원시작,청원인원,Link
0,저출산 관련 육아휴직 및 어린이집 증설,2018-10-31,3,https://www1.president.go.kr/petitions/427334
1,저출산 ↔ 어린이집 부족. 서로 너무 상충되는 관계 아닌가요?,2018-10-31,7,https://www1.president.go.kr/petitions/427316
2,어린이집을 유지해주세요,2018-10-31,476,https://www1.president.go.kr/petitions/426902
3,오늘 안하면 늦습니다. 누구나 마음놓고 출산할 수 있는 제도 마련 시급합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426884
4,가족돌봄휴가제도를 조속히 시행해야 합니다.,2018-10-31,2,https://www1.president.go.kr/petitions/426830


In [6]:
df.to_csv('data/petition_crawling_low_birth_rate.csv', sep=',', encoding='UTF-8')

In [7]:
! dir data

 C 드라이브의 볼륨: BOOTCAMP
 볼륨 일련 번호: 5415-72D5

 C:\Users\JuhnnyKo\Dropbox\Anaconda_src\data 디렉터리

2018-11-14  오전 02:55    <DIR>          .
2018-11-14  오전 02:55    <DIR>          ..
2018-11-14  오전 01:45    <DIR>          .ipynb_checkpoints
2018-11-07  오후 02:29             7,686 chicagomag_top_50_sandwiches.csv
2018-11-07  오후 05:00            10,314 chicagomag_top_50_sandwiches_ver2.csv
2018-10-30  오전 09:32               310 company.csv
2018-11-11  오후 06:22            43,781 evernote_schedule_2018.txt
2018-11-01  오후 04:23                62 girlgroup.json
2018-10-29  오후 04:32                21 hello.txt
2018-11-01  오후 04:26               162 member.json
2018-10-30  오전 11:06                10 mydata3.txt
2018-10-30  오전 11:14                 5 mydata4.txt
2018-10-30  오전 11:31                 8 mydata5.txt
2018-11-09  오전 11:10             8,872 naver_mv_ranking.csv
2018-11-01  오후 04:41               456 note_181101.txt
2018-11-06  오전 09:23             1,140 opengov_2018_seoul_cctv_status_by_yea